In [1]:
import nltk
import torch
from sklearn.metrics.pairwise import cosine_similarity
#from transformers import AutoModel, AutoTokenizer
from transformers import BertTokenizer, BertForSequenceClassification 
import pandas as pd
model_name = "bert-base-uncased"
model_path = "./results/checkpoint-5500"
#tokenizer = BertTokenizer.from_pretrained(model_path, local_files_only=True)
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_path, local_files_only=True)

In [2]:
for i in range(1):
    df_csv = pd.read_csv('D:/reviewsData/filtered{}.csv'.format(i)).drop(['Unnamed: 0'], axis=1)
    if i > 0:
        df_csv.index = df_csv.index + len(df)
        df = pd.concat([df,df_csv])
    else:
        df = df_csv

In [74]:
reviews = df['review'][df['app_name'] == 'The Witcher 3: Wild Hunt']
reviews = reviews[reviews.str.len() > 10].values.tolist()

In [75]:
reviews[0]

"One of the best RPG's of all time, worthy of any collection"

In [76]:
with torch.no_grad():
    character = model(torch.tensor([tokenizer.encode('art')]),output_hidden_states =True)['hidden_states'][-1][:,1,:]
token_frequency = {}
token_feature = {}
for i in range(len(reviews)):
    #words = nltk.word_tokenize(reviews[i])
    words = reviews[i]
    #noun_tags = ('NN','NNS','NNP')
    #word_pos = nltk.pos_tag(words)
    #for word in word_pos:
        #if pos in noun_tags:
    
    #print(i, words)
    input_ids = torch.tensor([tokenizer.encode(words, truncation=True, max_length=512)])

    tokens = tokenizer.tokenize(words)
    with torch.no_grad():
        #if len(input_ids.size()) != 0:
        features = model(input_ids,output_hidden_states=True)['hidden_states'][-1]
        for j in range(features.shape[1]-2):
            if tokens[j] not in token_frequency:
                token_frequency[tokens[j]] = 1
                token_feature[tokens[j]] = features[0,1+j,:]
            else:
                token_frequency[tokens[j]] += 1
                token_feature[tokens[j]] += features[0,1+j,:]
            #print(tokens[j], cosine_similarity(features[:,1+j,:], character))


In [77]:
for k in token_feature.keys():
    token_feature[k] = token_feature[k] / token_frequency[k]

In [80]:
len(token_feature)

20161

In [82]:
import pickle
a_file = open("token_feature.pkl", "wb")
pickle.dump(token_feature, a_file)
a_file.close()
a_file = open("token_frequency.pkl", "wb")
pickle.dump(token_frequency, a_file)
a_file.close()

In [ ]:
a_file = open("data.pkl", "rb")
output = pickle.load(a_file)
print(output)

In [78]:
token_feature_list = token_feature.values()
#token_feature_list = torch.FloatTensor(token_feature_list)
token_feature_list = np.array([item.cpu().detach().numpy() for item in token_feature_list])

TypeError: new(): data must be a sequence (got dict_values)

In [ ]:
token_frequency[k] > 10

In [45]:
from sklearn.cluster import KMeans
import numpy as np
kmeans = KMeans(n_clusters=5, random_state=0).fit(torch.FloatTensor(token_feature_list))

[1 1 2 1 1 2 1 1 1 2 1 1 1 1 2 1 2 1 1 1 3 1 1 1 1 2 1 1 1 4 1 1 1 1 1 1 1
 0 0 2 4 4 4 2 4 2 1 1 1 1 1 2 1 3 3 3 2 3 3 3 2 1 1 1 2 3 3 1 1 1 1 2 1 1
 2 4 4 4 4 2 4 2 4 4 2 4 4 4 2 2 4 4 4 4 4 4 4 4 2 2 2 4 4 4 4 4 2 4 4 2 4
 4 0 0 0 4 4 4 4 4 4 0 4 0 0 4 2 4 2 0 0 0]


In [54]:
#for i in range(max(kmeans.labels_)):
#[kmeans.labels_ == 0]
df_token = pd.DataFrame(np.array([list(token_frequency.keys()),kmeans.labels_]).transpose(), columns = ['tokens', 'group'])
df_token[df_token['group'] == '0']

,tokens,group
37,why,0
38,wouldn,0
112,use,0
113,same,0
114,sword,0
121,killing,0
123,repairs,0
124,craft,0
129,my,0
130,thing,0


In [55]:
df_token[df_token['group'] == '1']

,tokens,group
0,one,1
1,of,1
3,best,1
4,rpg,1
6,s,1
7,all,1
8,time,1
10,worthy,1
11,any,1
12,collection,1


In [46]:
word = "Geralt"
input_ids = torch.tensor([tokenizer.encode(word)])
with torch.no_grad():
    features = model(input_ids,output_hidden_states =True)
    print(word)
    print(cosine_similarity(features['hidden_states'][-1][:,1,:], character))

Geralt
[[0.24491188]]
